<a href="https://colab.research.google.com/github/ragul-20070603/GEN-AI-/blob/main/AI_Resume_Screener_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  AI Resume Screener with RAG and OCR
This notebook implements an AI-powered Resume Screener. It uses:

- **Google Colab** as backend server
- **Open source APIs** for skill extraction and RAG (Retrieval Augmented Generation)
- **OCR** for reading PDF/TXT files
- **Flask** for API hosting
- **Postman** for testing

### Features:
1. Upload resume in PDF/TXT format
2. Enter a job role (e.g., 'Web Developer')
3. Use RAG to fetch trending skills for that role
4. Use OCR to extract resume content
5. Output:
   - Skill match rate
   - Missing/in-demand skills


In [ ]:
# Install required libraries
!pip install openai flask flask-cors langchain pypdf pytesseract pdf2image python-docx transformers
!apt-get install -y poppler-utils tesseract-ocr

In [ ]:
# Helper: Extract text from PDF and TXT
import os
import pytesseract
from pdf2image import convert_from_path

def extract_text_from_file(file_path):
    if file_path.endswith(".txt"):
        with open(file_path, "r", encoding='utf-8') as f:
            return f.read()
    elif file_path.endswith(".pdf"):
        images = convert_from_path(file_path)
        text = ""
        for img in images:
            text += pytesseract.image_to_string(img)
        return text
    return ""

In [ ]:
# 🧠 Use RAG with OpenAI + LangChain (or HuggingFace API)
!pip install -U langchain-community
!pip install faiss-cpu chromadb
!pip install -U langchain openai

from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI

import openai
openai.api_key = "OPEN AI API KEY"  # Replace with your key

def get_trending_skills(job_role):
    prompt = f"List top 10 current trending skills for the role of {job_role}."
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100
    )
    return response.choices[0].text.strip().split("\n")

In [ ]:
# 📊 Match Skills
def compare_skills(resume_text, job_skills):
    matched = [skill for skill in job_skills if skill.lower() in resume_text.lower()]
    unmatched = [skill for skill in job_skills if skill not in matched]
    match_rate = len(matched) / len(job_skills) * 100
    return match_rate, matched, unmatched

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Replace YOUR_TOKEN with the token you copied
!ngrok config add-authtoken NGROK AUTH KEY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://ee1fc0432851.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
# 🚀 Flask API to serve the logic
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/screen', methods=['POST'])
def screen_resume():
    job_role = request.form['job_role']
    file = request.files['resume']

    file_path = f"/content/{file.filename}"
    file.save(file_path)
    resume_text = extract_text_from_file(file_path)

    skills = get_trending_skills(job_role)
    match_rate, matched, unmatched = compare_skills(resume_text, skills)

    return jsonify({
        'job_role': job_role,
        'match_rate': match_rate,
        'matched_skills': matched,
        'missing_skills': unmatched
    })

# To run in colab
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Jul/2025 18:11:09] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jul/2025 18:11:10] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jul/2025 18:16:37] "GET /screen HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [24/Jul/2025 18:32:12] "GET /screen HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [24/Jul/2025 19:30:00] "GET /screen%0A HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jul/2025 19:31:05] "GET /screen%0A?job%20role=web%20developer&resume%20=value HTTP/1.1" 404 -
ERROR:__main__:Exception on /screen [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dis